In [1]:
import time

import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.utils.data as data

%matplotlib inline
from matplotlib_inline.backend_inline import set_matplotlib_formats
from matplotlib.colors import to_rgba
from tqdm.notebook import tqdm  # Progress bar

set_matplotlib_formats("svg", "pdf")

# The Basics of PyTorch

In [2]:
print("Using torch", torch.__version__)

Using torch 1.11.0


- 통계적 함수에서 재현 가능하도록 random seed를 고정할 수 있음

In [3]:
torch.manual_seed(42)

## Tensor
### 초기화

In [4]:
x = torch.Tensor(2, 3, 4)  # 메모리에 있는 값으로 초기화 됨
print(x)

tensor([[[3.7661e-26, 4.5692e-41, 3.7661e-26, 4.5692e-41],
         [       nan,        nan, 1.5835e-43, 0.0000e+00],
         [3.7661e-26, 4.5692e-41, 3.7661e-26, 4.5692e-41]],

        [[0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 4.6243e-44, 0.0000e+00]]])




- `torch.zeros`: 0으로 채움
- `torch.ones`: 1로 채움
- `torch.rand`: 0과 1 사이의 유니폼 분포 샘플로 채움
- `torch.randn`: 평균 0, 표준편차 1의 정규분포 샘플로 채움
- `torch.arange`: N, N+1, N+2, ..., M 값으로 채움
- `torch.Tensor` (input list):  list의 요소 값으로 채움

In [6]:
x = torch.Tensor([[1, 2], [3, 4]])
print(x)

tensor([[1., 2.],
        [3., 4.]])


In [7]:
x = torch.rand(2, 3, 4)
print(x)

tensor([[[0.8823, 0.9150, 0.3829, 0.9593],
         [0.3904, 0.6009, 0.2566, 0.7936],
         [0.9408, 0.1332, 0.9346, 0.5936]],

        [[0.8694, 0.5677, 0.7411, 0.4294],
         [0.8854, 0.5739, 0.2666, 0.6274],
         [0.2696, 0.4414, 0.2969, 0.8317]]])


### Shape

In [8]:
shape = x.shape
print("Shape:", x.shape)

size = x.size()
print("Size:", size)

dim1, dim2, dim3 = x.size()
print("Size:", dim1, dim2, dim3)

Shape: torch.Size([2, 3, 4])
Size: torch.Size([2, 3, 4])
Size: 2 3 4


### Tensor to Numpy, and Numpy to Tensor

In [9]:
np_arr = np.array([[1, 2], [3, 4]])
tensor = torch.from_numpy(np_arr)

print("Numpy array:", np_arr)
print("PyTorch tensor:", tensor)

Numpy array: [[1 2]
 [3 4]]
PyTorch tensor: tensor([[1, 2],
        [3, 4]])


In [10]:
tensor = torch.arange(4)
np_arr = tensor.numpy()

print("PyTorch tensor:", tensor)
print("Numpy array:", np_arr)

PyTorch tensor: tensor([0, 1, 2, 3])
Numpy array: [0 1 2 3]


In [13]:
tensor = torch.arange(4)
np_arr = tensor.cpu().numpy()

print("PyTorch tensor:", tensor)  # CPU
print("Numpy array:", np_arr)

PyTorch tensor: tensor([0, 1, 2, 3])
Numpy array: [0 1 2 3]


### Operations
- numpy에 있는 연산자는 대부분 pytorch에도 포함. 모든 리스트는 [PyTorch documentation](https://pytorch.org/docs/stable/tensors.html#) 에서 확인할 수 있음

In [14]:
x1 = torch.rand(2, 3)
x2 = torch.rand(2, 3)
y = x1 + x2

print("X1", x1)
print("X2", x2)
print("Y", y)

X1 tensor([[0.1053, 0.2695, 0.3588],
        [0.1994, 0.5472, 0.0062]])
X2 tensor([[0.9516, 0.0753, 0.8860],
        [0.5832, 0.3376, 0.8090]])
Y tensor([[1.0569, 0.3448, 1.2448],
        [0.7826, 0.8848, 0.8151]])


- In-place operations 

In [19]:
x1 = torch.rand(2, 3)
x2 = torch.rand(2, 3)
print("X1 (before)", x1)
print("X2 (before)", x2)

x2.add_(x1)  # In-place operations 
print("X1 (after)", x1)
print("X2 (after)", x2)

X1 (before) tensor([[0.7539, 0.1952, 0.0050],
        [0.3068, 0.1165, 0.9103]])
X2 (before) tensor([[0.6440, 0.7071, 0.6581],
        [0.4913, 0.8913, 0.1447]])
X1 (after) tensor([[0.7539, 0.1952, 0.0050],
        [0.3068, 0.1165, 0.9103]])
X2 (after) tensor([[1.3979, 0.9024, 0.6632],
        [0.7981, 1.0078, 1.0550]])


- dimension 조절하기

In [16]:
x = torch.arange(6)
print("X", x)

X tensor([0, 1, 2, 3, 4, 5])


In [17]:
x = x.view(2, 3)  # Changing dimension
print("X", x)

X tensor([[0, 1, 2],
        [3, 4, 5]])


In [18]:
x = x.permute(1, 0)  # Swapping dimension 0 and 1
print("X", x)

X tensor([[0, 3],
        [1, 4],
        [2, 5]])


#### 행렬을 곱하는 다양한 방법
- `torch.matmul`: 두 텐서의 행렬 곱을 수행하며 차원에 따라 동작이 달라짐, 2차원 텐서 간의 곱인 경우 표준 행렬 곱과 같음. 더 높은 차원의 곱을 위해서 [브로드캐스팅](https://pytorch.org/docs/stable/generated/torch.matmul.html?highlight=matmul#torch.matmul)을 지원하며, numpy와 같이 a@b로 표현할 수도 있음 
- `torch.mm`: 행렬 곱을 수행하지만 브로드캐스팅을 지원하지는 않음
- `torch.bmm`: 3차원 텐서를 입력값으로 받으며 3차원 내 2차원 tensor 간의 곱을 batch 식으로 연산함. $(b \times n \times m)$ tensor @ $(b \times m \times p)$ tensor = $(b \times n \times p)$ tensor [(document)](https://pytorch.org/docs/stable/generated/torch.bmm.html?highlight=torch%20bmm#torch.bmm)
- `torch.einsum`: Einstein summation convention

In [22]:
x = torch.arange(6)
x = x.view(2, 3)
print("X", x)

X tensor([[0, 1, 2],
        [3, 4, 5]])


In [23]:
W = torch.arange(9).view(3, 3)
print("W", W)

W tensor([[0, 1, 2],
        [3, 4, 5],
        [6, 7, 8]])


In [24]:
h = torch.matmul(x, W)
print("h", h)

h tensor([[15, 18, 21],
        [42, 54, 66]])


### Dynamic Computation Graph and Backpropagation

생성된 텐서의 기본값은 gradients를 지원하지 않으나, `requires_grad_()` 함수를 사용하여 바꾸거나 생성할 때 `requires_grad=True` 옵션을 설정하여 gradients 지원하게 할 수 있음

In [25]:
x = torch.ones((3,))
print(x.requires_grad)

False


In [26]:
x.requires_grad_(True)
print(x.requires_grad)

True


$$ y = \frac{1}{|x|}\sum_i \left[(x_i + 2)^2 + 3\right] $$
위 함수에서 y 값의 최대 혹은 최소 값을 구해야한다면 x를 파라미터로 생각할 수 있음. 이 경우 gradient $ \partial y / \partial \mathbf{x} $ 을 얻어야 하며 아래는 입력 값으로 \mathbf{x}=[0,1,2] 를 가정하겠음

In [31]:
x = torch.arange(3, dtype=torch.float32, requires_grad=True)
print("X", x)

X tensor([0., 1., 2.], requires_grad=True)


In [32]:
a = x + 2
b = a ** 2
c = b + 3
y = c.mean()
print("Y", y)

Y tensor(12.6667, grad_fn=<MeanBackward0>)
